<img src="cryotempo_logo.png" alt="logo" width="200"/> <img src="esa_logo.png" alt="esa" width="170"/> <img src="earthwave_logo.png" alt="earthwave" width="150"/> <img src="UoE_logo.png" alt="uoe" width="200"/>

##<strong>*This is a Jupyter notebook that demonstrates how to download and use Cryotempo-EOLIS data, downloaded from cs2eo.org. Here, we will read in some gridded data and then plot DEMs*</strong> 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

To run this notebook, you will need to make sure that the folllowing packages are installed in your python environment (all can be installed via pip/conda)

    - matplotlib: for plotting
    - datetime: for handling timestamps
    - contextily: for adding basemaps to plots
    - specklia: for downloading data
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [ ]:
!pip install contextily
!pip install specklia

###<strong>1) Download Data:

Regardless of whether you are using the Google Colab environment, or have downloaded this notebook to your local drive, you will first need to download some data. You can use this notebook to plot any CryoTEMPO-EOLIS gridded data that you choose. For a quick example, follow the below instructions to download a small example dataset.</strong>

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Imports necessary to run the rest of the code

In [28]:
import datetime
import contextily as ctx
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from shapely import Polygon
from specklia import Specklia
plt.rcParams.update({'font.size': 14})

<strong>The following cells download gridded product data for Iceland from Specklia, Earthwave's geospatial point cloud database.</strong>

First, we will instantiate our Specklia Python client. To run the code, you will need to generate your own Specklia API key.

In [29]:
user_api_key = input('Please generate your own key using https://specklia.earthwave.co.uk/ApiKeys and paste it here:')
specklia_client = Specklia(user_api_key)

Next, we define our area of interest and query Specklia for data:

In [ ]:
iceland_polygon = Polygon(([-26., 63.], [-26., 68.], [-11., 68.], [-11., 63.], [-26., 63.]))

dataset_name = 'CryoTEMPO-EOLIS Gridded Product'
available_datasets = specklia_client.list_datasets()
gridded_product_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

gridded_product_data, sources = specklia_client.query_dataset(
    dataset_id=gridded_product_dataset['dataset_id'],
    epsg4326_polygon=iceland_polygon,
    min_datetime=datetime.datetime(2023,11,1),
    max_datetime=datetime.datetime(2023,11,30,23,59,59))

print(
    f'Query complete, {len(gridded_product_data)} points returned, drawn from {len(sources)} original sources.')


Next, we can check the metadata for the gridded product sources. In the output of this command, we can see lots of additional information assosciated with the gridded product files, including projection information, product version and support information.

In [ ]:
sources[0]

We can use the source information to determine the geospatial projection of the x and y columns:

In [ ]:
projections = []
for i, source in enumerate(sources):
    projection_for_source = source["source_information"]["geospatial_projection"]
    print(f'X and Y columns in source # {i+1} are in projection: {projection_for_source}')
    projections.append(projection_for_source)

if len(set(projections)) == 1:
    print('All the sources use the same projection! We will need this later on.')
    geospatial_projection = projections[0]
else:
    raise ValueError('You queried an area that covers multiple EOLIS regions - the X and Y columns in data returned by your query are not in the same projection. Please run a different query or reproject before continuing!')

If we run the cell below, we can see that the variables available for analysis include a time, x and y position, elevation and an associated uncertainty value. For more information about the derivation of these variables, see the CryoTEMPO-EOLIS ATBD, available at https://cryotempo-eolis.org/product-description/

In [ ]:
print('Variables returned by query: ', gridded_product_data.columns)

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
###<strong>2) Use the Cryotempo-EOLIS gridded product to plot a selection of DEMS, and show the difference in elevation for a specific time window</strong>

Below, we plot the 2-dimensional elevation data for the Vatnajokull glacier.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))
scatter = ax.scatter(gridded_product_data['x'],gridded_product_data['y'], c=gridded_product_data['elevation'], cmap='viridis', vmin=0, vmax=2000, s=20)
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')

# Add color bar
cbar = plt.colorbar(scatter)
cbar.set_label('Elevation [m]')
plt.title(f"CryoTEMPO-EOLIS DEM for ICELAND, {str(datetime.datetime.fromtimestamp(gridded_product_data['timestamp'].iloc[0]).date())}")

# set high resolution background map for zoomed in plot of Vatnajokull
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=7, crs=geospatial_projection)

# add thumbnail to corner of plot to show where the data is
thumbnail_ax = inset_axes(ax, width="25%", height=1.5, loc='upper left')

# set extent of iceland for second axis in EPSG:3413
thumbnail_ax.set_ylim(-2700000, -2200000)
thumbnail_ax.set_xlim(900000, 1500000)
thumbnail_ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)
for spine in ['bottom', 'top', 'right', 'left']:
    thumbnail_ax.spines[spine].set_color('red')
  
c = thumbnail_ax.scatter(gridded_product_data['x'],gridded_product_data['y'], c=gridded_product_data['elevation'], s=1, cmap='viridis')

# set lower resolution background map for zoomed out plot of iceland
ctx.add_basemap(thumbnail_ax, source=ctx.providers.Esri.WorldImagery, zoom=5, crs=geospatial_projection, attribution=False)